In [1]:
import polars as pl
import optuna
import lightgbm as lgb
import numpy as np

from src.util.constants import DATA_PATH, META_MODEL_PERFORMANCE, FIXED_LGB_PARAMETERS
from src.util.common import load_from_pickle, mean_grouped_spearman_correlation, save_as_pickle

/Users/jonas-data-science/Library/Caches/pypoetry/virtualenvs/nmr-DUJvlELt-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
selected_features = load_from_pickle(DATA_PATH / 'results/selected_features.pkl')
required_columns = ['era', 'target'] + selected_features

df_train_list = []
df_validate_list = []
for fold in range(2):
    df_train_fold = pl.read_parquet(f"{DATA_PATH}/folds/df_train_{fold}.parquet")
    df_validate_fold = pl.read_parquet(f"{DATA_PATH}/folds/df_validate_{fold}.parquet")

    df_train_fold = df_train_fold.select(required_columns)
    df_validate_fold = df_validate_fold.select(required_columns)

    df_train_list.append(df_train_fold)
    df_validate_list.append(df_validate_fold)
    del df_train_fold, df_validate_fold


df_meta_model = pl.read_parquet(f'{DATA_PATH}/folds/df_meta_model.parquet')
df_meta_model = df_meta_model.select(required_columns + ['numerai_meta_model'])

In [8]:
def performance_approximation(model: lgb.Booster, fold: int) -> tuple[float, float, float, float]:
    df_validate_fold = df_validate_list[fold]

    corr = mean_grouped_spearman_correlation(
        pl.Series(model.predict(df_validate_fold[selected_features].to_numpy())),
        df_validate_fold['target'],
        df_validate_fold['era']
    )

    df_mm_corr = pl.DataFrame({
        'prediction': pl.Series(model.predict(df_meta_model[selected_features].to_numpy())),
        'numerai_meta_model': df_meta_model['numerai_meta_model'],
        'era': df_meta_model['era']
    })
    corr_w_mm = df_mm_corr.select(
        pl.corr("prediction", "numerai_meta_model", method="spearman")
        .over('era', mapping_strategy='explode')
    ).mean().item()

    mmc_approximation = corr - corr_w_mm * META_MODEL_PERFORMANCE[fold]
    performance = .75 * corr + 2.25 * mmc_approximation

    return performance, corr, corr_w_mm, mmc_approximation

In [9]:
def objective(trial: optuna.trial.Trial) -> float:
    # we use these exponents to create a log scale that includes zero
    min_sum_hessian_in_leaf_exponent = trial.suggest_int("min_sum_hessian_in_leaf_exponent", 0, 10)
    lambda_l1_exponent = trial.suggest_int("lambda_l1_exponent", 0, 8)
    lambda_l2_exponent = trial.suggest_int("lambda_l2_exponent", 0, 12)
    parameter_sample = {
        "learning_rate": trial.suggest_float("learning_rate", .001, .2, log=True),
        "num_boost_round": trial.suggest_int("num_boost_round", 10, 5000),
        'max_bin': trial.suggest_int('max_bin', 2 ** 3 - 1, 2 ** 10 - 1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2 ** 2 - 1, 2 ** 12 - 1, log=True),
        "bagging_fraction": trial.suggest_float("bagging_fraction", .1, 1.0),
        "feature_fraction": trial.suggest_float("feature_fraction", .1, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 500000, log=True),
        "min_sum_hessian_in_leaf": (10 ** min_sum_hessian_in_leaf_exponent - 1) / 10 ** 5,
        "lambda_l1": (10 ** lambda_l1_exponent - 1) / 10 ** 5,
        "lambda_l2": (10 ** lambda_l2_exponent - 1) / 10 ** 5
    }
    # use default for max_depth = -1 (tree complexity is regularised via num_leaves)

    parameters = {
        **FIXED_LGB_PARAMETERS,
        **parameter_sample
    }

    result_performance = []

    for fold in range(2):
        lgb_train = lgb.Dataset(
            df_train_list[fold][selected_features].to_numpy(),
            label=df_train_list[fold]['target'].to_numpy()
        )

        model = lgb.train(
            params=parameters,
            train_set=lgb_train,
            num_boost_round=parameters['num_boost_round']
        )

        performance, corr, corr_w_mm, mmc_approximation = performance_approximation(model, fold)
        trial.set_user_attr(f"performance_{fold}", performance)
        trial.set_user_attr(f"correlation_{fold}", corr)
        trial.set_user_attr(f"correlation_with_meta_model_{fold}", corr_w_mm)
        trial.set_user_attr(f"mmc_approximation_{fold}", mmc_approximation)
        print(f'Results for fold {fold}: Performance {performance:.5f}, correlation: {corr:.5f}, correlation with meta model: {corr_w_mm:.5f}, MMC (approx.): {mmc_approximation:.5f}')

        # stop trial if it does not show promise (by default, median pruner is used)
        trial.report(performance, step=fold)
        if fold == 0 and trial.number > 10 and trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        result_performance.append(performance)

    return np.mean(result_performance)

In [10]:
study = optuna.create_study(direction="maximize")

[I 2026-01-23 16:30:36,063] A new study created in memory with name: no-name-12f77bfe-7681-4383-bcc4-82ab250c8500


In [ ]:
# cumulative performance 50 runs:
# best performance: 0.0496
# runtime: 21:20 h

In [13]:
study.optimize(objective, n_trials=50, show_progress_bar=True)

  0%|          | 0/45 [00:00<?, ?it/s]

Results for fold 0: Performance 0.04243, correlation: 0.03336, correlation with meta model: 0.58235, MMC (approx.): 0.00774


Best trial: 2. Best value: 0.048259:   2%|▏         | 1/45 [15:52<11:38:25, 952.40s/it]

Results for fold 1: Performance 0.03278, correlation: 0.02662, correlation with meta model: 0.56551, MMC (approx.): 0.00570
[I 2026-01-23 22:35:50,047] Trial 5 finished with value: 0.037603236700830024 and parameters: {'min_sum_hessian_in_leaf_exponent': 9, 'lambda_l1_exponent': 1, 'lambda_l2_exponent': 5, 'learning_rate': 0.0053180346343605625, 'num_boost_round': 1070, 'max_bin': 172, 'num_leaves': 65, 'bagging_fraction': 0.805090587506008, 'feature_fraction': 0.8623755926495538, 'min_data_in_leaf': 12570}. Best is trial 2 with value: 0.048258982529888944.
Results for fold 0: Performance 0.00837, correlation: 0.01726, correlation with meta model: 0.43860, MMC (approx.): -0.00204


Best trial: 2. Best value: 0.048259:   4%|▍         | 2/45 [21:27<7:02:11, 589.10s/it] 

...

Results for fold 1: Performance 0.04002, correlation: 0.03010, correlation with meta model: 0.60387, MMC (approx.): 0.00775
[I 2026-01-24 16:50:01,956] Trial 47 finished with value: 0.04611078977601342 and parameters: {'min_sum_hessian_in_leaf_exponent': 9, 'lambda_l1_exponent': 3, 'lambda_l2_exponent': 8, 'learning_rate': 0.029802813406551235, 'num_boost_round': 4603, 'max_bin': 234, 'num_leaves': 1600, 'bagging_fraction': 0.6375989838151461, 'feature_fraction': 0.40272650042938396, 'min_data_in_leaf': 92223}. Best is trial 45 with value: 0.04964793617421635.


Best trial: 45. Best value: 0.0496479:  98%|█████████▊| 44/45 [18:30:46<18:15, 1095.44s/it]

Results for fold 0: Performance 0.01094, correlation: 0.01757, correlation with meta model: 0.42178, MMC (approx.): -0.00099
[I 2026-01-24 16:50:44,043] Trial 48 pruned. 


Best trial: 45. Best value: 0.0496479: 100%|██████████| 45/45 [18:36:48<00:00, 1489.08s/it]

Results for fold 0: Performance 0.02190, correlation: 0.02372, correlation with meta model: 0.49756, MMC (approx.): 0.00183
[I 2026-01-24 16:56:46,193] Trial 49 pruned. 


In [14]:
# visualise result
for param in study.best_params.keys():
    optuna.visualization.plot_slice(study, params=[param]).show()

In [15]:
save_as_pickle(study, DATA_PATH / 'results/study_lgb.pkl')